Загрузка библиотек.

In [18]:
import pandas as pd
import plotly.express as px
import requests
from bs4 import BeautifulSoup

Загрузка листов excel файла. 

In [19]:
df_articles = pd.read_excel('Аналитик данных _Тестовое задание _РБК.xlsx'
                            , sheet_name = 'База публикаций')
df_views = pd.read_excel('Аналитик данных _Тестовое задание _РБК.xlsx'
                         , sheet_name = 'Выгрузка'
                         , skiprows = lambda x: x in [0, 1])

Просмотр данных, их типов, пропущенных значений.

In [20]:
df_articles.head()

,ID публикации,Заголовок материала,URL
0,5bf52faf9a79477da426e81b,Как обанкротиться подешевле: прагматичные наст...,https://pro.rbc.ru/news/5bf52faf9a79477da426e81b
1,5c04ee439a7947b3fe48fe55,Имитация тунца: как искусственная рыба вытесня...,https://pro.rbc.ru/news/5c04ee439a7947b3fe48fe55
2,5c04f06c9a7947b48d48fe60,Сколько россияне тратят на путешествия,https://pro.rbc.ru/news/5c04f06c9a7947b48d48fe60
3,5c0695d69a79472e0096479e,Самые ходовые: какие специалисты в строительст...,https://pro.rbc.ru/news/5c0695d69a79472e0096479e
4,5c07891f9a794775e61affd1,Как запустить новый продукт: четыре обязательн...,https://pro.rbc.ru/news/5c07891f9a794775e61affd1


In [21]:
df_articles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 454 entries, 0 to 453
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID публикации        454 non-null    object
 1   Заголовок материала  454 non-null    object
 2   URL                  454 non-null    object
dtypes: object(3)
memory usage: 10.8+ KB


In [22]:
df_articles.isnull().count()

ID публикации          454
Заголовок материала    454
URL                    454
dtype: int64

In [23]:
df_views.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2696 entries, 0 to 2695
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Адрес страницы  2696 non-null   object
 1   Просмотры       2696 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 42.2+ KB


In [24]:
df_views.describe()

,Просмотры
count,2696.000000
mean,106.040430
std,333.579372
min,1.000000
25%,4.000000
50%,11.000000
75%,34.250000
max,2849.000000


In [25]:
df_views.isnull().count()

Адрес страницы    2696
Просмотры         2696
dtype: int64

Типы данных верные, пропущенных значений нет.

Исключение из списка ненужных адресов.

In [26]:
URL = {'test.pro',
'test.v2.pro',
'feature-rbcnews',
'/preview/',
'staging.pro',
'staging.v2.pro'}

In [27]:
for i in URL:
    df_views = df_views.loc[~df_views['Адрес страницы'].str.contains(i)]
df_views

,Адрес страницы,Просмотры
0,https://pro.rbc.ru/news/5c4af9ed9a7947bd94fe0cc9,2849
1,http://pro.rbc.ru/news/5c61b2d39a794798b96fab0c,2845
2,https://pro.rbc.ru/news/5c6195799a79478b57ba6cf3,2819
3,https://pro.rbc.ru//news/5c63fbff9a7947481bf4ce1b,2736
4,https://pro.rbc.ru/news/5c46d39b9a7947fc20fbcce6,2704
...,...,...
2691,https://pro.rbc.ru/news/5c63fbff9a7947481bf4ce1b,1
2692,https://pro.rbc.ru/news/5c6297279a7947d83339d513,1
2693,https://pro.rbc.ru/news/5c62c45c9a7947eaedb915e4,1
2694,https://pro.rbc.ru/news/5c643a319a794760bbe62974,1


Группировка по адресам ссылок и подсчет общего количества просмотров, сортировка по убыванию просмотров.

In [28]:
df_views = df_views.pivot_table(
    values = 'Просмотры'
    , index = 'Адрес страницы'
    , aggfunc = 'sum'
).sort_values('Просмотры', ascending = False).reset_index()
df_views.head(20)

,Адрес страницы,Просмотры
0,https://pro.rbc.ru/news/5c658e479a79474d32847233,11320
1,https://pro.rbc.ru/news/5c65950d9a79474f82f780d7,7422
2,https://pro.rbc.ru/news/5c503adb9a794784a693903b,7044
3,https://pro.rbc.ru/news/5c6454df9a794769a732e049,6483
4,https://pro.rbc.ru/news/5c643a319a794760bbe62974,6325
5,https://pro.rbc.ru/news/5c6579039a794742acb7ebe3,5656
6,https://pro.rbc.ru/news/5c60c9fd9a794747ed04cfe2,5471
7,https://pro.rbc.ru//news/5c63fbff9a7947481bf4ce1b,5365
8,https://pro.rbc.ru/news/5c627bba9a7947cb36ac9898,5277
9,https://pro.rbc.ru/news/5c6195799a79478b57ba6cf3,5276


Соединение двух таблиц для получения общего списка с заголовками, адресами и просмотрами.  
Сохранение 20-ти самых посещаемых.

In [29]:
df_20 = df_views.merge(df_articles, how = 'left', left_on = 'Адрес страницы', right_on = 'URL')\
[['Адрес страницы', 'Заголовок материала', 'Просмотры']].head(20)
df_20

,Адрес страницы,Заголовок материала,Просмотры
0,https://pro.rbc.ru/news/5c658e479a79474d32847233,"Глава BlaBlaCar — РБК: «Мы оказывали услуги, н...",11320
1,https://pro.rbc.ru/news/5c65950d9a79474f82f780d7,«Алые паруса» сворачиваются: почему владельцы ...,7422
2,https://pro.rbc.ru/news/5c503adb9a794784a693903b,Недооцененный экспорт: что имеет смысл продава...,7044
3,https://pro.rbc.ru/news/5c6454df9a794769a732e049,Требования работодателя vs ожидания кандидатов...,6483
4,https://pro.rbc.ru/news/5c643a319a794760bbe62974,Восемь способов увеличить средний чек,6325
5,https://pro.rbc.ru/news/5c6579039a794742acb7ebe3,Менеджеры РФ получают дополнительное образован...,5656
6,https://pro.rbc.ru/news/5c60c9fd9a794747ed04cfe2,Как трансформируется логистика в ретейле: лучш...,5471
7,https://pro.rbc.ru//news/5c63fbff9a7947481bf4ce1b,NaN,5365
8,https://pro.rbc.ru/news/5c627bba9a7947cb36ac9898,Эксперты: российскому рынку кофеен можно «попр...,5277
9,https://pro.rbc.ru/news/5c6195799a79478b57ba6cf3,Математика российского люкса: точный портрет п...,5276


К некоторым ссылкам нет заголовков на странице 'База публикаций'.  
Можно найти заголовки на страницах по ссылкам и добавить.

In [30]:
def find_title_by_url(URL):
    return BeautifulSoup(requests.get(URL).text, 'lxml').find('h1').text.strip()

In [31]:
df_20['Заголовок материала'].fillna(df_20['Адрес страницы'].apply(find_title_by_url), inplace=True)
df_20 = df_20[['Адрес страницы', 'Заголовок материала', 'Просмотры']]
df_20

,Адрес страницы,Заголовок материала,Просмотры
0,https://pro.rbc.ru/news/5c658e479a79474d32847233,"Глава BlaBlaCar — РБК: «Мы оказывали услуги, н...",11320
1,https://pro.rbc.ru/news/5c65950d9a79474f82f780d7,«Алые паруса» сворачиваются: почему владельцы ...,7422
2,https://pro.rbc.ru/news/5c503adb9a794784a693903b,Недооцененный экспорт: что имеет смысл продава...,7044
3,https://pro.rbc.ru/news/5c6454df9a794769a732e049,Требования работодателя vs ожидания кандидатов...,6483
4,https://pro.rbc.ru/news/5c643a319a794760bbe62974,Восемь способов увеличить средний чек,6325
5,https://pro.rbc.ru/news/5c6579039a794742acb7ebe3,Менеджеры РФ получают дополнительное образован...,5656
6,https://pro.rbc.ru/news/5c60c9fd9a794747ed04cfe2,Как трансформируется логистика в ретейле: лучш...,5471
7,https://pro.rbc.ru//news/5c63fbff9a7947481bf4ce1b,Кедры решают все: как вывести косметику на аме...,5365
8,https://pro.rbc.ru/news/5c627bba9a7947cb36ac9898,Эксперты: российскому рынку кофеен можно «попр...,5277
9,https://pro.rbc.ru/news/5c6195799a79478b57ba6cf3,Математика российского люкса: точный портрет п...,5276


Добавление столбца с кликабельными ссылками.

In [32]:
def make_clickable(URL, name):
    return f'<a href="{URL}" target="_blank">{name}</a>'

In [33]:
pd.options.mode.chained_assignment = None
df_20['ТОП 20'] = df_20.apply(lambda x: make_clickable(x['Адрес страницы'], x['Заголовок материала']), axis = 1)
df_20[['ТОП 20']].style.hide(axis="index").format({'url': make_clickable})

ТОП 20
"Глава BlaBlaCar — РБК: «Мы оказывали услуги, но ничего не зарабатывали»"
«Алые паруса» сворачиваются: почему владельцы сети выходят из бизнеса
Недооцененный экспорт: что имеет смысл продавать за рубеж
Требования работодателя vs ожидания кандидатов в ТЭКе — 2019
Восемь способов увеличить средний чек
Менеджеры РФ получают дополнительное образование в основном за свой счет
Как трансформируется логистика в ретейле: лучшие практики
Кедры решают все: как вывести косметику на американский рынок
Эксперты: российскому рынку кофеен можно «поправиться» на 870 млрд руб.
Математика российского люкса: точный портрет покупателя предметов роскоши


Визуализация результата.  
ТОП-20 самых посещаемых страниц с кликабельными ссылками и столбчатой диаграммой, отображающей количество посещений.

In [34]:
fig = px.bar(df_20.sort_values('Просмотры', ascending = True), x = 'Просмотры', y = 'ТОП 20', text_auto = True)
fig.show()